<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#iris-dataset" data-toc-modified-id="iris-dataset-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>iris dataset</a></span></li><li><span><a href="#score" data-toc-modified-id="score-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>score</a></span></li><li><span><a href="#Hyper-Parameter" data-toc-modified-id="Hyper-Parameter-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Hyper Parameter</a></span><ul class="toc-item"><li><span><a href="#3-way-holdout" data-toc-modified-id="3-way-holdout-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>3-way holdout</a></span></li><li><span><a href="#GridSearchCV" data-toc-modified-id="GridSearchCV-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>GridSearchCV</a></span></li></ul></li></ul></div>

# [AI 오픈 이노베이션] 기본 저녁반<a class="tocSkip">
GitHub link: [https://github.com/realblack0/8th_ai_lecture_fundamental](https://github.com/realblack0/8th_ai_lecture_fundamental)  
E-Mail: realblack0@gmail.com

## iris dataset

In [4]:
from sklearn.datasets import load_iris

In [5]:
data = load_iris()

In [3]:
import pandas as pd

In [6]:
data_pd = pd.DataFrame(data.data, columns=data.feature_names)
target_pd = pd.DataFrame(data.target, columns=['target'])
iris = pd.concat([data_pd, target_pd], axis=1)

In [7]:
iris

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [8]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
 4   target             150 non-null    int32  
dtypes: float64(4), int32(1)
memory usage: 5.4 KB


## score

- 대충의 성능 파악(dummy)

- 실제 모델은 내가 가진 모든 데이터로 만든다. 

In [9]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(iris.iloc[:, :-1], iris.target)

In [24]:
from sklearn.metrics import confusion_matrix

In [25]:
from sklearn.neighbors import KNeighborsClassifier

In [26]:
knn = KNeighborsClassifier()

In [27]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [28]:
# 행은 실제, 열은 예측한 값 이다.
# 실제 0인데, 0으로 예측한 값 12개
# 실제 1인데, 1로 예측한 값 9개 & 2로 예측한 값 1개
# 실제 2인데, 1로 예측한 값 1개 & 2로 예측한 값 15개
confusion_matrix(y_test, knn.predict(X_test))

array([[12,  0,  0],
       [ 0,  9,  1],
       [ 0,  1, 15]], dtype=int64)

In [16]:
# 0, 1, 2는 다음과 같다. 
data.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

위의 confusion matirx는 다음과 같다.

|   |setosa로 예측|versicolor로 예측|virginica로 예측|
|---|---|---|---|
|실제 setosa|12|0|0|
|실제 versicolor|0|9|1|
|실제 virginica|0|1|15|

---

이진 분류하는 confusion matrix에서 각 위치를 다음과 같이 이름 붙인다.

|||Predict||
|---|---|---|---|
|   |    |True|False|
|Actual|True|True Positive|False Negative|
|   |False|False Positive|True Negative|

- False Positivie는 1종 오류  
- False Negative는 2종 오류

1종 오류와 2종 오류 중에서 어떤 것이 더 중요한지는, 의사결정자의 판단에 따라 다르다.

---



- accuracy(정확도)

    전체 데이터 중에 예측이 맞은 비율

    $$accuracy = \frac{TP+TN}{TP+FP+TN+FN}$$

- precision(정밀도)

    양성으로 예측한 것 중에서 양성이 맞는 비율

    즉, 예측한 거 중에 맞은 비율

    $$precision = \frac{TP}{TP+FP}$$

- recall(재현률)

    실제 양성인 것  중에서 양성으로 예측한 비율

    즉, 맞은 비율

    $$reacll = \frac{TP}{TP+FN}$$

multi class 일 경우는, 각 클래스 별로 precision을 구하고 평균을 내면 최종 precision이 된다.  
최종 recall도 마찬가지 방법으로 구한다.

In [29]:
from sklearn.model_selection import cross_val_score

In [30]:
from sklearn.metrics import recall_score

In [31]:
# accuracy 대신 recall로 cross validation score를 측정할 수 있다.
cross_val_score(KNeighborsClassifier(), X_train, y_train, scoring='recall_micro')

array([1.        , 1.        , 0.95454545, 0.95454545, 0.95454545])

In [32]:
from sklearn.metrics import classification_report

In [33]:
# 내부적으로 confusion matrix를 만든 후 평가 지표별로 점수를 계산해준다.
print(classification_report(y_test, knn.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.90      0.90      0.90        10
           2       0.94      0.94      0.94        16

    accuracy                           0.95        38
   macro avg       0.95      0.95      0.95        38
weighted avg       0.95      0.95      0.95        38



## Hyper Parameter

하이퍼 파라미터에 따라서 성능이 달라진다.  
고로, 최적의 하이퍼 파라미터를 찾는 것이 중요하다.

In [34]:
# 단순하게 하이퍼 파라미터를 for문으로 찾는 방법을 생각할 수 있다.
temp = []
for i in range(1, 20):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    temp.append(knn.score(X_test, y_test))

In [35]:
temp

[0.9473684210526315,
 0.9473684210526315,
 0.9473684210526315,
 0.9473684210526315,
 0.9473684210526315,
 0.9473684210526315,
 0.9210526315789473,
 0.9473684210526315,
 0.9210526315789473,
 0.9210526315789473,
 0.9473684210526315,
 0.9210526315789473,
 0.9473684210526315,
 0.9473684210526315,
 0.9473684210526315,
 0.9210526315789473,
 0.9473684210526315,
 0.8947368421052632,
 0.8947368421052632]

In [36]:
import numpy as np

In [37]:
# argmax는 가장 큰 값의 index를 알려준다. 
# n_neighbors가 1일때 성능이 가장 좋다. 
np.argmax(temp)

0

### 3-way holdout

데이터를 총 3등분한다. 
- train set : 모델 학습에 사용한다.
- validationt set : 하이퍼 파라미터를 찾기 위해서 사용한다.
- test set : 한번도 안 본 데이터로 모델의 성능을 측정하기 위해서 사용한다.

In [38]:
X_train, X_test, y_train, y_test = train_test_split(iris.iloc[:, :-1], iris.target)

In [39]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train)

In [43]:
# 하이퍼 파라미터를 찾기 위해서 3-way holdout을 한다.
temp = []
for i in range(1, 20):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    temp.append(knn.score(X_validation, y_validation))

In [44]:
np.argmax(temp)

0

In [45]:
# 이 방법은 모든 데이터를 사용하기 때문에 위의 temp 방식 보다 많이 사용한다. 
temp2 = []
for i in range(1, 20):
    temp2.append(cross_val_score(KNeighborsClassifier(n_neighbors=i), iris.iloc[:,:-1], iris.target).mean())

In [46]:
# temp와 최적의 하이퍼 파라미터가 다르게 나왔다. 
np.argmax(temp2)

5

In [47]:
temp2

[0.96,
 0.9466666666666665,
 0.9666666666666668,
 0.9733333333333334,
 0.9733333333333334,
 0.9800000000000001,
 0.9800000000000001,
 0.9666666666666668,
 0.9733333333333334,
 0.9800000000000001,
 0.9800000000000001,
 0.9800000000000001,
 0.9733333333333334,
 0.9666666666666666,
 0.9666666666666668,
 0.9666666666666668,
 0.9666666666666668,
 0.9666666666666666,
 0.9666666666666668]

In [48]:
# 최종 모델은 모든 데이터를 써서 학습한다.
knn_final = KNeighborsClassifier(n_neighbors=6)
knn_final.fit(iris.iloc[:, :-1], iris.target)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                     weights='uniform')

### GridSearchCV

scikit-learn에서 하이퍼 파라미터를 편하게 찾는 기능을 제공하고 있다. 

In [49]:
from sklearn.model_selection import GridSearchCV

In [58]:
knn = KNeighborsClassifier()

In [60]:
# scikit-learn의 모델들은 get_params를 공통적으로 가지고 있다.
# 알고리즘이 사용할 수 있는 하이퍼 파라미터를 알려준다.
knn.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [50]:
# param_grid의 key는 get_params에 나오는 하이퍼 파라미터만 쓸 수 있다.
# param_grid의 value는 테스트해볼 하이퍼 파라미터를 리스트로 넣는다.
grid = GridSearchCV(KNeighborsClassifier(), {'n_neighbors':[2,3,4,5,6,7]})

In [52]:
grid.fit(iris.iloc[:, :-1], iris.target)

GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': [2, 3, 4, 5, 6, 7]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [53]:
# 가장 좋은 하이퍼 파라미터를 한번에 찾아준다.
grid.best_params_

{'n_neighbors': 6}

In [54]:
# 가장 좋은 하이퍼 파라미터의 성능을 알려준다.
grid.best_score_

0.9800000000000001

In [57]:
# grid search한 결과를 총 정리해준다.
grid.cv_results_

{'mean_fit_time': array([0.00259261, 0.0028038 , 0.00340114, 0.00339971, 0.0036006 ,
        0.00259995]),
 'std_fit_time': array([0.00049645, 0.00038461, 0.00049274, 0.00149542, 0.00049095,
        0.00048998]),
 'mean_score_time': array([0.00399919, 0.00440502, 0.00459146, 0.00459957, 0.00439868,
        0.00400052]),
 'std_score_time': array([0.00109671, 0.00101165, 0.00135298, 0.00149641, 0.00102036,
        0.00063196]),
 'param_n_neighbors': masked_array(data=[2, 3, 4, 5, 6, 7],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_neighbors': 2},
  {'n_neighbors': 3},
  {'n_neighbors': 4},
  {'n_neighbors': 5},
  {'n_neighbors': 6},
  {'n_neighbors': 7}],
 'split0_test_score': array([0.96666667, 0.96666667, 0.96666667, 0.96666667, 0.96666667,
        0.96666667]),
 'split1_test_score': array([0.93333333, 0.96666667, 0.96666667, 1.        , 1.        ,
        1.        ]),
 'split2_test_score': array([0

In [56]:
# pandas로 만들면 보기 쉽다.
# column이 너무 많을때는 전치(T)해서 보면 편하다.
pd.DataFrame(grid.cv_results_).T

,0,1,2,3,4,5
mean_fit_time,0.00259261,0.0028038,0.00340114,0.00339971,0.0036006,0.00259995
std_fit_time,0.000496446,0.000384607,0.000492736,0.00149542,0.000490953,0.00048998
mean_score_time,0.00399919,0.00440502,0.00459146,0.00459957,0.00439868,0.00400052
std_score_time,0.00109671,0.00101165,0.00135298,0.00149641,0.00102036,0.000631962
param_n_neighbors,2,3,4,5,6,7
params,{'n_neighbors': 2},{'n_neighbors': 3},{'n_neighbors': 4},{'n_neighbors': 5},{'n_neighbors': 6},{'n_neighbors': 7}
split0_test_score,0.966667,0.966667,0.966667,0.966667,0.966667,0.966667
split1_test_score,0.933333,0.966667,0.966667,1,1,1
split2_test_score,0.933333,0.933333,0.966667,0.933333,0.966667,0.966667
split3_test_score,0.9,0.966667,0.966667,0.966667,0.966667,0.966667


In [61]:
# 하이퍼 파라미터를 여러개 사용할 수 있다.
grid = GridSearchCV(KNeighborsClassifier(), {'n_neighbors':[2,3,4,5,6,7], 'weights':['distance', 'uniform']})

In [62]:
grid.fit(iris.iloc[:, :-1], iris.target)

GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': [2, 3, 4, 5, 6, 7],
                         'weights': ['distance', 'uniform']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [63]:
# params를 보면 하이퍼 파라미터들의 가능한 조합을 모두 계산했다. 
pd.DataFrame(grid.cv_results_).T

,0,1,2,3,4,5,6,7,8,9,10,11
mean_fit_time,0.00280075,0.00260053,0.00220118,0.00299983,0.00260005,0.00300012,0.00219831,0.00259514,0.00340018,0.0031981,0.00320091,0.00279636
std_fit_time,0.000399736,0.000800598,0.000402046,0.000631731,0.00080066,0.000631582,0.000398958,0.000479017,0.000502843,0.000980559,0.00074893,0.000743541
mean_score_time,0.00220037,0.00319891,0.00220089,0.00379953,0.00220008,0.00440893,0.00198145,0.00418992,0.00259848,0.0040009,0.0021986,0.00360026
std_score_time,0.000400188,0.00039897,0.00039969,0.000747972,0.000400096,0.000783867,2.25264e-05,0.000750529,0.000801423,0.000897951,0.000400711,0.000800327
param_n_neighbors,2,2,3,3,4,4,5,5,6,6,7,7
param_weights,distance,uniform,distance,uniform,distance,uniform,distance,uniform,distance,uniform,distance,uniform
params,"{'n_neighbors': 2, 'weights': 'distance'}","{'n_neighbors': 2, 'weights': 'uniform'}","{'n_neighbors': 3, 'weights': 'distance'}","{'n_neighbors': 3, 'weights': 'uniform'}","{'n_neighbors': 4, 'weights': 'distance'}","{'n_neighbors': 4, 'weights': 'uniform'}","{'n_neighbors': 5, 'weights': 'distance'}","{'n_neighbors': 5, 'weights': 'uniform'}","{'n_neighbors': 6, 'weights': 'distance'}","{'n_neighbors': 6, 'weights': 'uniform'}","{'n_neighbors': 7, 'weights': 'distance'}","{'n_neighbors': 7, 'weights': 'uniform'}"
split0_test_score,0.966667,0.966667,0.966667,0.966667,0.966667,0.966667,0.966667,0.966667,0.966667,0.966667,0.966667,0.966667
split1_test_score,0.966667,0.933333,0.966667,0.966667,0.966667,0.966667,1,1,1,1,1,1
split2_test_score,0.933333,0.933333,0.933333,0.933333,0.933333,0.966667,0.9,0.933333,0.966667,0.966667,0.966667,0.966667


In [1]:
from sklearn.datasets import load_wine

In [2]:
data = load_wine()

In [3]:
import pandas as pd

/Users/sangkim/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [4]:
data_pd = pd.DataFrame(data.data, columns = data.feature_names)

In [5]:
data_pd['target'] = pd.DataFrame(data.target, columns = ['target'])

In [6]:
wine = data_pd.copy()

In [7]:
wine

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


In [8]:
from sklearn.model_selection import GridSearchCV

In [9]:
from sklearn.linear_model import LogisticRegression

In [12]:
from sklearn.neighbors import KNeighborsClassifier

In [14]:
def all_models(X_train, y_train, X_validation, y_validation, cv=10):
    #1 Logistic Regression
    grid_lr = GridSearchCV(LogisticRegression(), cv=cv)
    grid_lr.fit(X_train, y_train)
    lr_pd = pd.DataFrame(grid_lr.cv_results_)
    # 2. knn
    grid_knn = GridSearchCV(KNeighborsClassifier(), cv=cv)
    grid_knn.fit(X_train, y_train)
    knn_pd = pd.DataFrame(grid_knn.cv_results_)

In [16]:
grid = GridSearchCV(KNeighborsClassifier(), [{'n_neighbors':[2,3,4,5]},{'n_neighbors':[2,3,4,5],'weights':['uniform','distance']}])

In [18]:
grid.fit(wine.iloc[:,:-1], wine.target)

GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'n_neighbors': [2, 3, 4, 5]},
                         {'n_neighbors': [2, 3, 4, 5],
                          'weights': ['uniform', 'distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [20]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001884,0.000242,0.002658,0.000463,2,NaN,{'n_neighbors': 2},0.611111,0.611111,0.638889,0.657143,0.800000,0.663651,0.070381,11
1,0.001519,0.000064,0.002014,0.000040,3,NaN,{'n_neighbors': 3},0.638889,0.694444,0.666667,0.657143,0.857143,0.702857,0.079204,5
2,0.001458,0.000152,0.001990,0.000133,4,NaN,{'n_neighbors': 4},0.638889,0.694444,0.638889,0.657143,0.771429,0.680159,0.049941,9
3,0.001456,0.000135,0.001959,0.000062,5,NaN,{'n_neighbors': 5},0.722222,0.666667,0.638889,0.657143,0.771429,0.691270,0.048780,7
4,0.001325,0.000099,0.001919,0.000156,2,uniform,"{'n_neighbors': 2, 'weights': 'uniform'}",0.611111,0.611111,0.638889,0.657143,0.800000,0.663651,0.070381,11
5,0.001216,0.000056,0.001086,0.000173,2,distance,"{'n_neighbors': 2, 'weights': 'distance'}",0.805556,0.638889,0.666667,0.685714,0.828571,0.725079,0.076912,1
6,0.001255,0.000135,0.001864,0.000209,3,uniform,"{'n_neighbors': 3, 'weights': 'uniform'}",0.638889,0.694444,0.666667,0.657143,0.857143,0.702857,0.079204,5
7,0.001244,0.000106,0.001076,0.000137,3,distance,"{'n_neighbors': 3, 'weights': 'distance'}",0.722222,0.722222,0.666667,0.657143,0.857143,0.725079,0.071393,1
8,0.001268,0.000176,0.001827,0.000242,4,uniform,"{'n_neighbors': 4, 'weights': 'uniform'}",0.638889,0.694444,0.638889,0.657143,0.771429,0.680159,0.049941,9
9,0.001321,0.000143,0.001062,0.000133,4,distance,"{'n_neighbors': 4, 'weights': 'distance'}",0.666667,0.694444,0.694444,0.657143,0.857143,0.713968,0.073113,4


In [21]:
from sklearn.pipeline import Pipeline

In [23]:
from sklearn.preprocessing import StandardScaler

In [25]:
pipe = Pipeline([('ss', StandardScaler()), ('knn', KNeighborsClassifier())])

In [27]:
vars(pipe)

{'steps': [('ss', StandardScaler(copy=True, with_mean=True, with_std=True)),
  ('knn',
   KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                        metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                        weights='uniform'))],
 'memory': None,
 'verbose': False}

In [30]:
pipe.steps[-1][-1].get_params

<bound method BaseEstimator.get_params of KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')>

In [36]:
grid = GridSearchCV(pipe, {'knn__n_neighbors':[2,3,4,5]}) ## pipe는 언더바 2개 __ 넣어줘야 된다?

In [37]:
grid.fit(wine.iloc[:,:-1], wine.target)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('ss',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('knn',
                                        KNeighborsClassifier(algorithm='auto',
                                                             leaf_size=30,
                                                             metric='minkowski',
                                                             metric_params=None,
                                                             n_jobs=None,
                                                             n_neighbors=5, p=2,
                                                             weights='uniform'))],
                                verbose=False),
            

In [38]:
pipe = Pipeline([('ss', StandardScaler()), ('clf', KNeighborsClassifier())])

In [39]:
from sklearn.linear_model import LogisticRegression

In [40]:
grid = GridSearchCV(pipe, {'clf':[KNeighborsClassifier(), LogisticRegression()]})

In [41]:
grid.fit(wine.iloc[:,:-1], wine.target)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('ss',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('clf',
                                        KNeighborsClassifier(algorithm='auto',
                                                             leaf_size=30,
                                                             metric='minkowski',
                                                             metric_params=None,
                                                             n_jobs=None,
                                                             n_neighbors=5, p=2,
                                                             weights='uniform'))],
                                verbose=False),
            

In [43]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002863,0.000375,0.002604,0.000391,"KNeighborsClassifier(algorithm='auto', leaf_si...","{'clf': KNeighborsClassifier(algorithm='auto',...",0.944444,0.944444,0.972222,1.000000,0.885714,0.949365,0.037911,2
1,0.014453,0.013450,0.001059,0.000523,"LogisticRegression(C=1.0, class_weight=None, d...","{'clf': LogisticRegression(C=1.0, class_weight...",0.972222,0.972222,1.000000,0.971429,1.000000,0.983175,0.013741,1


In [49]:
grid = GridSearchCV(pipe, 
                    [{'clf':[KNeighborsClassifier()], 'clf__n_neighbors':[2,3,4,5]}, 
                     {'clf':[LogisticRegression()]}])

In [50]:
grid.fit(wine.iloc[:,:-1], wine.target)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('ss',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('clf',
                                        KNeighborsClassifier(algorithm='auto',
                                                             leaf_size=30,
                                                             metric='minkowski',
                                                             metric_params=None,
                                                             n_jobs=None,
                                                             n_neighbors=5, p=2,
                                                             weights='uniform'))],
                                verbose=False),
            

In [52]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_clf__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002952,0.000269,0.002651,0.000219,"KNeighborsClassifier(algorithm='auto', leaf_si...",2,"{'clf': KNeighborsClassifier(algorithm='auto',...",0.916667,0.944444,0.944444,1.000000,0.914286,0.943968,0.030875,3
1,0.002361,0.000164,0.002224,0.000214,"KNeighborsClassifier(algorithm='auto', leaf_si...",3,"{'clf': KNeighborsClassifier(algorithm='auto',...",0.888889,0.944444,0.972222,1.000000,0.914286,0.943968,0.039630,3
2,0.002338,0.000082,0.002192,0.000134,"KNeighborsClassifier(algorithm='auto', leaf_si...",4,"{'clf': KNeighborsClassifier(algorithm='auto',...",0.888889,0.944444,0.972222,1.000000,0.914286,0.943968,0.039630,3
3,0.002363,0.000095,0.002216,0.000088,"KNeighborsClassifier(algorithm='auto', leaf_si...",5,"{'clf': KNeighborsClassifier(algorithm='auto',...",0.944444,0.944444,0.972222,1.000000,0.885714,0.949365,0.037911,2
4,0.007885,0.000320,0.000881,0.000148,"LogisticRegression(C=1.0, class_weight=None, d...",NaN,"{'clf': LogisticRegression(C=1.0, class_weight...",0.972222,0.972222,1.000000,0.971429,1.000000,0.983175,0.013741,1
